In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sentence_transformers import SentenceTransformer
import pandas as pd
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA


In [2]:
df = pd.read_csv(r'E:\Omdena\Cleaned_data.csv')

In [3]:
df.head()

,UID,PRODUCT_NAME,IMAGE_URL,PRODUCT_PRICE,PRODUCT_LINK,PRODUCT_INFORMATION_T,CATEGORY,SUB_CATEGORY,PRICE_PER_KG/L,PRODUCT_QUANTITY,...,STORAGE_INFORMATION,PRODUCT_REVIEWS,STORE_LINK,STORE_NAME,PRICE_PER_KG/L_UNIT,ENERGY_KJ,SUB_SUB_CATEGORY,PRODUCT_BRAND,PRODUCT_INFORMATION,MANUFACTURER
0,1,ODENWÄLDER LEBENSMITTEL - All our groceries ar...,https://m.media-amazon.com/images/I/61UMMiMnCB...,NaN,https://www.amazon.de/ODENW%C3%84LDER-LEBENSMI...,NaN,Food & Beverage,Plant Based,NaN,NaN,...,NaN,2 Sternebewertungen,NaN,Amazon,NaN,NaN,butter,Odenwälder Lebensmittel,NaN,Odenwälder Lebensmittel  Made im Odenwald UG
1,2,Yerba Mate Tea Set: Rosamonte Tradicional 1kg ...,https://m.media-amazon.com/images/I/71+OkCm0i3...,NaN,https://www.amazon.de/Yerba-Mate-Tee-Set-Antio...,NaN,Food & Beverage,Coffee & Tea,NaN,NaN,...,NaN,1 Sternebewertung,NaN,Amazon,NaN,NaN,Mate tea,Besuche den Yerbox-Store,NaN,Yerbox / Hreniuk S.A. (Rosamonte)
2,3,Yerba Mate Tea Set: Rosamonte Traditional 1 kg...,https://m.media-amazon.com/images/I/71QjU48wfH...,NaN,https://www.amazon.de/Yerba-Mate-Tee-Set-Mates...,NaN,Food & Beverage,Coffee & Tea,NaN,NaN,...,NaN,15 Sternebewertungen,NaN,Amazon,NaN,NaN,Mate tea,Besuche den Yerbee-Store,NaN,Rosamonte / HREÑUK S.A.
3,4,1000g Raw Puer Tea Iceland Ancient Tree Puer T...,https://m.media-amazon.com/images/I/71zYFtabeL...,NaN,https://www.amazon.de/%E9%80%9A%E7%94%A8-%C3%8...,NaN,Food & Beverage,Coffee & Tea,NaN,NaN,...,NaN,NaN,NaN,Amazon,NaN,NaN,dark tea,NaN,NaN,NaN
4,5,1000g Yunnan Classic Ripe Puer Tea Organic Pu...,https://m.media-amazon.com/images/I/71B-EZsheC...,NaN,https://www.amazon.de/%E9%80%9A%E7%94%A8-Gekoc...,NaN,Food & Beverage,Coffee & Tea,NaN,NaN,...,NaN,NaN,NaN,Amazon,NaN,NaN,dark tea,NaN,NaN,NaN


In [4]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_text(series):
    # Tokenize the text into words
    tokens = series.apply(lambda text: word_tokenize(text.lower()))
    
    # Remove punctuation
    tokens = tokens.apply(lambda words: [word for word in words if word not in string.punctuation])
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = tokens.apply(lambda words: [word for word in words if word not in stop_words])
    
    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    tokens = tokens.apply(lambda words: [lemmatizer.lemmatize(word) for word in words])
    
    # Join the tokens back into a single string
    preprocessed_text = tokens.apply(lambda words: ' '.join(words))
    
    return preprocessed_text


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
df.columns

Index(['UID', 'PRODUCT_NAME', 'IMAGE_URL', 'PRODUCT_PRICE', 'PRODUCT_LINK',
       'PRODUCT_INFORMATION_T', 'CATEGORY', 'SUB_CATEGORY', 'PRICE_PER_KG/L',
       'PRODUCT_QUANTITY', 'SATURATED_FATTY_ACIDS', 'CARBOHYDRATES', 'SUGAR',
       'PROTEIN', 'FIBER', 'SALT', 'FAT', 'INGREDIENTS', 'ALLERGENS',
       'STORAGE_INFORMATION', 'PRODUCT_REVIEWS', 'STORE_LINK', 'STORE_NAME',
       'PRICE_PER_KG/L_UNIT', 'ENERGY_KJ', 'SUB_SUB_CATEGORY', 'PRODUCT_BRAND',
       'PRODUCT_INFORMATION', 'MANUFACTURER'],
      dtype='object')

In [6]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load and preprocess the product data
product_data = df
# Preprocess the product description or any other relevant text
product_data['processed_description'] = preprocess_text(product_data['CATEGORY'])

# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(product_data['processed_description'])

# Compute cosine similarity
similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)


In [7]:
def get_similar_products(index, similarity_matrix, product_data, top_n=5):
    # Get the similarity scores for the given index
    similarity_scores = similarity_matrix[index]
    
    # Get the indices of top N similar products
    top_indices = similarity_scores.argsort()[::-1][1:top_n+1]  # Exclude the product itself
    
    # Retrieve the similar products from the product_data DataFrame
    similar_products = product_data.iloc[top_indices]
    
    return similar_products


In [8]:
similarity_matrix.shape

(42942, 42942)

In [9]:
target_product_index = 156  # Index of the target product in product_data
similar_products = get_similar_products(target_product_index, similarity_matrix, product_data, top_n=5)
print("MAIN PRODUCT = " , product_data.iloc[target_product_index]['PRODUCT_NAME'])
print("*******************)")
print("SIMILAR PRODUCTS : ")
print(similar_products['PRODUCT_NAME'])


MAIN PRODUCT =  Aln. Bioland Oat Drink Vanilla 1 l
*******************)
SIMILAR PRODUCTS : 
14320    GEPA white tea Darjeeling, pack of 5 (5 x 50 g...
14318        Gepa Organic Earl Gray Black Tea 34g, 20 bags
14317       GEPA Indian Chai, 1 pack (1 x 100 g) - organic
14316              GEPA - coconut milk 200 ml / LK-BIO-149
14315                  Georgian Tschatscha Classic 45% Vol
Name: PRODUCT_NAME, dtype: object
